In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Request access_token
import requests
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cache-Control': 'no-cache',
}


# Fill with TikTok API credentials. Can be requested at https://developers.tiktok.com/products/research-api/
data = {
    'client_key': 'API_CREDENTIALS',
    'client_secret': 'API_CREDENTIALS',
    'grant_type': 'client_credentials',
}


response = requests.post('https://open.tiktokapis.com/v2/oauth/token/', headers=headers, data=data).json()
auth = response['access_token']

In [ ]:
auth

In [ ]:
# API request
def get_data(start_date, end_date, account):
    headers = {
        "authorization": f"bearer {auth}",
        "Content-Type": "application/json",
    }

    data = {
        "query": {
            "and": [
                { "operation": "IN", "field_name": "username", "field_values": [account] }
            ]
        },
        "start_date": start_date,
        "end_date": end_date,
        "max_count": 100
    }
    response = requests.post(
        "https://open.tiktokapis.com/v2/research/video/query/?fields=id,like_count,video_description,share_count,view_count,create_time,region_code,comment_count,voice_to_text,effect_ids",
        headers=headers, data=json.dumps(data)
    ).json()
    print(response)

    return response

# Saving each month of each political party into a csv file
def write_file(folder_path, filename, response):

    if 'data' in response and 'videos' in response['data']:
        videos = response['data']['videos'] #extracting video data
    else:
        print("No videos found in response data.")
        return

    os.makedirs(folder_path, exist_ok=True)  #folder to save csv
    filepath = os.path.join(folder_path, filename)

    # Finding keys to store the correct value for each column, defines headers of csv
    all_keys = set()
    for video in videos:
        all_keys.update(video.keys())

    fieldnames = sorted(all_keys)

    with open(filepath, 'w', newline='', encoding='utf-8') as csvFile: #writes data
        csvwriter = csv.DictWriter(csvFile, fieldnames=fieldnames, delimiter="\t")
        csvwriter.writeheader()
        for video in videos:
            csvwriter.writerow(video)

    print(f"Data saved to {filepath}")

In [ ]:
# Should be changed each time
account = 'ACCOUNT_NAME'
start_date = 'START_DATE'
end_date = 'END_DATE'
filename = 'FILENAME'

# Define the path to save the file
folder_path = "PARTY_PATH"

response = get_data(start_date, end_date, account)
write_file(folder_path, filename,response)